In [1]:
from pyspark.sql import SparkSession

spark = SparkSession.builder.appName('rec').getOrCreate()

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
24/01/21 21:55:46 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [2]:
from pyspark.ml.recommendation import ALS
from pyspark.ml.evaluation import RegressionEvaluator

/Users/schanlar/opt/anaconda3/lib/python3.9/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.26.2
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


In [3]:
data = spark.read.csv("movielens_ratings.csv", header=True, inferSchema=True)

In [4]:
data.printSchema()

root
 |-- movieId: integer (nullable = true)
 |-- rating: double (nullable = true)
 |-- userId: integer (nullable = true)



In [5]:
data.head(1)

[Row(movieId=2, rating=3.0, userId=0)]

In [6]:
data.describe().show()

+-------+------------------+------------------+------------------+
|summary|           movieId|            rating|            userId|
+-------+------------------+------------------+------------------+
|  count|              1501|              1501|              1501|
|   mean| 49.40572951365756|1.7741505662891406|14.383744170552964|
| stddev|28.937034065088994| 1.187276166124803| 8.591040424293272|
|    min|                 0|               1.0|                 0|
|    max|                99|               5.0|                29|
+-------+------------------+------------------+------------------+



In [7]:
train_data, test_data = data.randomSplit([0.8,0.2])

In [8]:
als = ALS(maxIter=5, regParam=0.01, userCol='userId', itemCol='movieId', ratingCol='rating')

In [9]:
model = als.fit(train_data)

24/01/21 21:55:57 WARN InstanceBuilder: Failed to load implementation from:dev.ludovic.netlib.blas.JNIBLAS
24/01/21 21:55:57 WARN InstanceBuilder: Failed to load implementation from:dev.ludovic.netlib.blas.VectorBLAS
24/01/21 21:55:57 WARN InstanceBuilder: Failed to load implementation from:dev.ludovic.netlib.lapack.JNILAPACK


In [10]:
predictions = model.transform(test_data)

In [11]:
predictions.show(5)

+-------+------+------+----------+
|movieId|rating|userId|prediction|
+-------+------+------+----------+
|      0|   1.0|    26|-0.8573231|
|      1|   1.0|    26|-1.8747495|
|      0|   1.0|    22| 0.7714779|
|      1|   1.0|     3|0.77582496|
|      1|   1.0|     5|0.59487134|
+-------+------+------+----------+
only showing top 5 rows



In [12]:
evaluator = RegressionEvaluator(metricName='rmse', labelCol='rating', predictionCol='prediction')

In [13]:
rmse = evaluator.evaluate(predictions)

In [14]:
print(f"RMSE : {rmse:.2f}")

RMSE : 1.76


In [15]:
single_user = test_data.filter(test_data['userId'] == 11).select(['userId', 'movieId'])

In [16]:
single_user.show()

+------+-------+
|userId|movieId|
+------+-------+
|    11|      0|
|    11|     10|
|    11|     18|
|    11|     19|
|    11|     21|
|    11|     47|
|    11|     64|
|    11|     75|
+------+-------+



In [17]:
recommendations = model.transform(single_user)

In [18]:
recommendations.orderBy('prediction', ascending=False).show()

+------+-------+----------+
|userId|movieId|prediction|
+------+-------+----------+
|    11|     10| 4.1071606|
|    11|     18|  3.799441|
|    11|     64| 3.4272218|
|    11|      0| 1.6636143|
|    11|     19| 1.2113746|
|    11|     75| 1.0520581|
|    11|     21|0.84820956|
|    11|     47|-1.3318553|
+------+-------+----------+

